# PREPARING ENVIRONMENT

In [ ]:
#Sometimes it requires authentication in HF to proceed dataset streaming modes.
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!git clone https://github.com/DrikDrik/Automatic-speech-recognition-with-DL

fatal: destination path 'Automatic-speech-recognition-with-DL' already exists and is not an empty directory.


In [3]:
import sys
sys.path.append('/content/Automatic-speech-recognition-with-DL/src')
sys.path.append('/content/Automatic-speech-recognition-with-DL/')

In [1]:
!pip install -r /content/Automatic-speech-recognition-with-DL/requirements.txt

# TRAIN

In [ ]:
!python Automatic-speech-recognition-with-DL/train.py \
    trainer.num_epochs=0 trainer.device=cuda dataloader.batch_size_train=64 pretrained.enabled=False # or True if you want to fine-tune from checkpoint
                                                                                                      # and trainer.start_epoch=X for jumping on checkpoint with reduced lr
### Model weights are saving automatically in /content/ folder in the end of every epoch
### prepare for wandb login

# INFERENCE

**Checkpoints are loading automatically, to disable pass skip_model_load=True**

## Example of run with greedy (argmax) approach

In [ ]:
!python Automatic-speech-recognition-with-DL/inference.py \
    output_dir= "inference_predictions" batch_size=8 datasets.hf.enabled=true strategy='greedy'

tokenizer_config.json: 283kB [00:00, 242MB/s]
vocab.json: 836kB [00:00, 89.0MB/s]
tokenizer.json: 2.48MB [00:00, 166MB/s]
merges.txt: 494kB [00:00, 99.5MB/s]
normalizer.json: 52.7kB [00:00, 105MB/s]
added_tokens.json: 34.6kB [00:00, 103MB/s]
special_tokens_map.json: 2.19kB [00:00, 13.9MB/s]
README.md: 11.0kB [00:00, 44.8MB/s]
Resolving data files: 100% 48/48 [00:00<00:00, 90.57it/s]
Resolving data files: 100% 48/48 [00:00<00:00, 89.70it/s]
Resolving data files: 100% 48/48 [00:00<00:00, 46689.84it/s]
Resolving data files: 100% 64/64 [00:00<00:00, 132.28it/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1UpX3_UgrbRTWYunAMHPsR09a1_zHzj7E
To: /content/encoder.pth
100% 6.29M/6.29M [00:00<00:00, 25.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?export=download&id=1A1Cb1TCn5LWYuIADkOsfvzlBBBi2L2bi
From (redirected): https://drive.google.com/uc?export=download&id=1A1Cb1TCn5LWYuIADkOsfvzlBBBi2L2bi&confirm=t&uuid=91a33325-363c-4168-b8d1-0ebd93009755
To:

## Example of run with beam-search approach

In [ ]:
!python Automatic-speech-recognition-with-DL/inference.py \
    output_dir= "inference_predictions" batch_size=8 datasets.hf.enabled=true strategy="beam"


Resolving data files: 100% 48/48 [00:00<00:00, 143.02it/s]
Resolving data files: 100% 48/48 [00:00<00:00, 135.48it/s]
Resolving data files: 100% 48/48 [00:00<00:00, 227.28it/s]
Resolving data files: 100% 64/64 [00:00<00:00, 280.70it/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1UpX3_UgrbRTWYunAMHPsR09a1_zHzj7E
To: /content/encoder.pth
100% 6.29M/6.29M [00:00<00:00, 29.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?export=download&id=1A1Cb1TCn5LWYuIADkOsfvzlBBBi2L2bi
From (redirected): https://drive.google.com/uc?export=download&id=1A1Cb1TCn5LWYuIADkOsfvzlBBBi2L2bi&confirm=t&uuid=4c00cf6a-9f64-4433-bd70-faa577cbca34
To: /content/decoder.pth
100% 97.7M/97.7M [00:01<00:00, 96.9MB/s]
Too many dataloader workers: 4 (max is dataset.num_shards=1). Stopping 3 dataloader workers.
[2025-10-16 13:18:40,653][datasets.iterable_dataset][WARNING] - Too many dataloader workers: 4 (max is dataset.num_shards=1). Stopping 3 dataloader workers.
test_clean (bea

## Downloading ground truth texts to demonstrate evaluation script in work


In [ ]:
import os
from datasets import load_dataset
from datasets import load_dataset, Audio, Features, Value

folders = {
    "test_clean": "ground_truth/test_clean",
    "test_other": "ground_truth/test_other"
}

for folder in folders.values():
    os.makedirs(folder, exist_ok=True)

features = Features(
        {
            "file": Value("string"),
            "audio": Audio(decode=False),
            "text": Value("string"),
            "speaker_id": Value("int64"),
            "chapter_id": Value("int64"),
            "id": Value("string"),
        }
    )

clean_ds_stream = load_dataset("librispeech_asr", "clean", split="test", features=features, streaming=True)
other_ds_stream = load_dataset("librispeech_asr", "other", split="test", features=features, streaming=True)

streams = {
    "test_clean": clean_ds_stream,
    "test_other": other_ds_stream
}

for folder_name, dataset_stream in streams.items():
    folder_path = folders[folder_name]

    for sample in dataset_stream:
        text = sample['text']
        if text.strip():
            file_id = sample['id']
            file_name = f"{folder_name}_ID{file_id}.txt"
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(text)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

## Evaluation script for cer and wer metrics. only needed to enter folder locations of prediction and target texts to run

In [ ]:
#TEST CLEAN
!python Automatic-speech-recognition-with-DL/calc_metrics.py \
    paths.gt_dir=/content/ground_truth/test_clean \
    paths.pred_dir=/content/inference_predictions/test_clean

Evaluated 2620 samples
WER = 0.1663 (16.63%)
CER = 0.0942 (9.42%)
Saved JSON report to metrics/wer_cer_report.json


In [ ]:
#TEST OTHER
!python Automatic-speech-recognition-with-DL/calc_metrics.py \
    paths.gt_dir=/content/ground_truth/test_other paths.pred_dir=/content/inference_predictions/test_other

Evaluated 2939 samples
WER = 0.3173 (31.73%)
CER = 0.1833 (18.33%)
Saved JSON report to metrics/wer_cer_report.json


## Downloading some dataset to demonsrate custom dataset class in work


In [ ]:
import os
from datasets import load_dataset, Audio, Features, Value
from tqdm import tqdm

base_dir = "/ds_test/"
num_samples_to_load = 2000

audio_dir = os.path.join(base_dir, "audio")
transcriptions_dir = os.path.join(base_dir, "transcriptions")

print(f"Creating directory: {audio_dir}")
os.makedirs(audio_dir, exist_ok=True)
print(f"Creating directory: {transcriptions_dir}")
os.makedirs(transcriptions_dir, exist_ok=True)

features = Features(
    {
        "id": Value("string"),
        "audio": Audio(decode=False),
        "duration_ms": Value("float32"),
        "text": Value("string"),
    }
)

print("Loading dataset stream...")
dataset = load_dataset(
    "MLCommons/peoples_speech",
    "clean",
    split="test",
    features=features,
    streaming=True,
    trust_remote_code=True
)

dataset_iterator = dataset.take(num_samples_to_load)

for example in tqdm(dataset_iterator, total=num_samples_to_load, desc="Saving files"):
    utterance_id = 'clean_ID' + example["id"] # THE "ID" PREFIX IS CRUCIAL! IT WON'T WORK WITHOUT IT ######!!!
    audio_data = example["audio"]
    audio_bytes = audio_data["bytes"]
    original_path = audio_data["path"]
    safe_utterance_id = utterance_id.replace("/", "_")
    _, file_extension = os.path.splitext(original_path)

    audio_filename = f"{safe_utterance_id}{file_extension}"
    audio_filepath = os.path.join(audio_dir, audio_filename)

    transcription_filename = f"{safe_utterance_id}.txt"
    transcription_filepath = os.path.join(transcriptions_dir, transcription_filename)

    with open(audio_filepath, "wb") as f_audio:
        f_audio.write(audio_bytes)

    with open(transcription_filepath, "w", encoding="utf-8") as f_text:
        f_text.write(example["text"])


print(f"\nSuccessfully saved {num_samples_to_load} audio and transcription files.")


`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'MLCommons/peoples_speech' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'MLCommons/peoples_speech' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Creating directory: /ds_test/audio
Creating directory: /ds_test/transcriptions
Loading dataset stream...


Resolving data files:   0%|          | 0/804 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/804 [00:00<?, ?it/s]

Saving files: 100%|██████████| 2000/2000 [00:04<00:00, 429.53it/s]


Successfully saved 2000 audio and transcription files.


## Demonstrating custom dir dataset in work


In [ ]:
!python Automatic-speech-recognition-with-DL/inference.py \
    batch_size=8 datasets.custom_dir.enabled=true datasets.custom_dir.path=/ds_test
# quality isn't so great there because this dataset has significantly longer audio lengths than LibriSpeech, anyways it is only for demonstration and might be changed for any other custom datset

Downloading...
From: https://drive.google.com/uc?export=download&id=1UpX3_UgrbRTWYunAMHPsR09a1_zHzj7E
To: /content/encoder.pth
100% 6.29M/6.29M [00:00<00:00, 17.8MB/s]
Downloading...
From (original): https://drive.google.com/uc?export=download&id=1A1Cb1TCn5LWYuIADkOsfvzlBBBi2L2bi
From (redirected): https://drive.google.com/uc?export=download&id=1A1Cb1TCn5LWYuIADkOsfvzlBBBi2L2bi&confirm=t&uuid=310888c9-2942-4891-b1cb-4834e619b061
To: /content/decoder.pth
100% 97.7M/97.7M [00:00<00:00, 116MB/s]
custom (beam5): 100% 250/250 [09:59<00:00,  2.40s/it]

=== custom FINAL: WER=0.6785, CER=0.4595 ===

    custom_wer_beam          : 0.6784763758120368
    custom_cer_beam          : 0.45946691075111107
    custom_sample_table      : [{'dataset': 'custom', 'batch_idx': 0, 'target': "you can't use it for a recreational facility", 'predict_beam': 'he is for recognition of the solidy', 'wer_beam': 1.0, 'cer_beam': 0.6590909090909091}, {'dataset': 'custom', 'batch_idx': 1, 'target': "and what that mean

## Demonstrating evaluation script for custom dataset

In [ ]:
!python Automatic-speech-recognition-with-DL/calc_metrics.py \
    paths.gt_dir=/ds_test/transcriptions \
    paths.pred_dir=/content/inference_predictions/custom

Evaluated 2000 samples
WER = 0.6785 (67.85%)
CER = 0.4595 (45.95%)
Saved JSON report to metrics/wer_cer_report.json


# **YOUR CUSTOM DATASET**

## Here you are able to enter the g-drive link for downloading your own dataset and run inference on it

Structure should be like this:

In [ ]:
# NameOfTheDirectoryWithUtterances
# ├── audio
# │   ├── UtteranceID1.wav # may be flac or mp3
# │   ├── UtteranceID2.wav
# │   .
# │   .
# │   .
# │   └── UtteranceIDn.wav
# └── transcriptions # ground truth, may not exist
#     ├── UtteranceID1.txt
#     ├── UtteranceID2.txt
#     .
#     .
#     .
#     └── UtteranceIDn.txt

Enter the link here:

In [ ]:
import gdown, zipfile, pathlib

zip_path = gdown.download(input('Enter download link: '), output=None, quiet=False)
zipfile.ZipFile(zip_path).extractall("/your_ds/")

Run for inference:

In [ ]:
!python Automatic-speech-recognition-with-DL/inference.py \
    batch_size=8 output_dir="inference_custom" datasets.custom_dir.enabled=true strategy='beam' datasets.custom_dir.path=/your_ds

Run for metrics:

In [ ]:
!python Automatic-speech-recognition-with-DL/calc_metrics.py \
    paths.gt_dir=/your_ds/transcriptions \
    paths.pred_dir=/content/inference_custom/custom